# Ноутбук для обучения ms-marco моделей на смержинных текстах

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Модели и датасеты, которые есть на этот момент

In [2]:
dataset_names = ['initial', '2 same topics', '2 different topics', '3 same topics', '3 different topics']
dataset_paths = ['/content/drive/MyDrive/QA_training/datasets_csv/ms_initial_train.csv',
                 '/content/drive/MyDrive/QA_training/merged datasets/ms_marco/merged_by_2_train.csv',
                 '/content/drive/MyDrive/QA_training/merged datasets/ms_marco/old_merged_datasets/ms_merged_train.csv',
                 '/content/drive/MyDrive/QA_training/merged datasets/ms_marco/merged_by_3_train.csv',
                 '/content/drive/MyDrive/QA_training/merged datasets/ms_marco/random_merged_by_3_train.csv']

In [3]:
model_names = ['bert', 'same topic bert', 'different topics bert']
model_paths = ['/content/drive/MyDrive/QA_training/models/initial_bert_model_1',
               '/content/drive/MyDrive/QA_training/models/msmarco_models/merged_by_2_bert',
               '/content/drive/MyDrive/QA_training/models/merged_bert_model_1']
model_v2_paths = ['/content/drive/MyDrive/QA_training/models/initial_bert_model_2',
                  '/content/drive/MyDrive/QA_training/models/msmarco_models/2_same_topics_ms_bert_2',
                  '/content/drive/MyDrive/QA_training/models/msmarco_models/2_different_topics_ms_bert_2']
model_v4_paths = ['/content/drive/MyDrive/QA_training/models/msmarco_models/inital_bert_model_3',
                  '/content/drive/MyDrive/QA_training/models/msmarco_models/3_same_topics_ms_bert_1.5',
                  '/content/drive/MyDrive/QA_training/models/msmarco_models/3_different_topics_ms_bert_1.5']

new_model_v2_names = ['inital_bert_model_2', '2_same_topics_ms_bert_2', '2_different_topics_ms_bert_2']
new_model_v3_names = ['inital_bert_model_3', '2_same_topics_ms_bert_3', '2_different_topics_ms_bert_3']
new_model_v4_names = ['inital_bert_model_3', '3_same_topics_ms_bert_1.5', '3_different_topics_ms_bert_1.5']
new_model_v5_names = ['inital_bert_model_3', '3_same_topics_ms_bert_3', '3_different_topics_ms_bert_3']



# Обучение

In [4]:
!pip install import-ipynb
%cd '/content/drive/My Drive/QA_training/notebooks'
import import_ipynb
from my_functions import *

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=3c75002e827c197a02b1f7151cf8560272c4fa35744d282bfb5b26a9cac76d01
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
/content/drive/My Drive/QA_training/notebooks
importing Jupyter notebook from my_functions.ipynb
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_3k70yx0
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-_3k70yx0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 7.7MB/s 
     |████████████████████████████████| 2.9MB 20.5MB/s 
  Created wheel for transformers: filename=transformers-4.2.0.dev0-cp36-none-any.whl size=1735733 sha256=2933145c66344c6d7813db710653929ab755ea4a90be545d9c6561131bcec9cb
  Store

In [5]:
#tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [6]:
number = 3

In [7]:
train_csv_path = dataset_paths[number]
print('Dataset path:', train_csv_path)


train_dataset = prepare_and_merge_data(train_csv_path, tokenizer, 'msmarco', text_num=3, parts_num = 2, part = 2)



Dataset path: /content/drive/MyDrive/QA_training/merged datasets/ms_marco/merged_by_3_train.csv


In [8]:
dir = "/content/drive/MyDrive/QA_training/models/msmarco_models/"+ new_model_v5_names[number-2]

print('Importing model path:', model_v4_paths[number-2])
print('New model path:', dir)
#model = BertForQuestionAnswering.from_pretrained(model_v4_paths[number-2])
#model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")
model =  BertForQuestionAnswering.from_pretrained(dir+'/checkpoint-36000')

Importing model path: /content/drive/MyDrive/QA_training/models/msmarco_models/3_same_topics_ms_bert_1.5
New model path: /content/drive/MyDrive/QA_training/models/msmarco_models/3_same_topics_ms_bert_3


In [9]:
training_args = TrainingArguments(
    output_dir=dir,          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    logging_dir=dir+'/logs',            # directory for storing logs
    logging_steps=250,
    prediction_loss_only=False,
    label_names = ["start_positions", "end_positions"],
    overwrite_output_dir = True,
    learning_rate = 3e-5
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    #eval_dataset=val_dataset,            # evaluation dataset

)

In [10]:
#trainer.train()
trainer.train(dir+'/checkpoint-36000')
trainer.save_model(dir)

Step,Training Loss
36250,0.008400
36500,1.183200
36750,1.204300
37000,1.221700
37250,1.199600
37500,1.239700
37750,1.173900
38000,1.195300
38250,1.194100
38500,1.185100
